# BERT (from HuggingFace Transformers) for Text Extraction

**Author:** [Apoorv Nandan](https://twitter.com/NandanApoorv)<br>
**Date created:** 2020/05/23<br>
**Last modified:** 2020/05/23<br>
**Description:** Fine tune pretrained BERT from HuggingFace Transformers on SQuAD.

## Introduction

This demonstration uses SQuAD (Stanford Question-Answering Dataset).
In SQuAD, an input consists of a question, and a paragraph for context.
The goal is to find the span of text in the paragraph that answers the question.
We evaluate our performance on this data with the "Exact Match" metric,
which measures the percentage of predictions that exactly match any one of the
ground-truth answers.

We fine-tune a BERT model to perform this task as follows:

1. Feed the context and the question as inputs to BERT.
2. Take two vectors S and T with dimensions equal to that of
   hidden states in BERT.
3. Compute the probability of each token being the start and end of
   the answer span. The probability of a token being the start of
   the answer is given by a dot product between S and the representation
   of the token in the last layer of BERT, followed by a softmax over all tokens.
   The probability of a token being the end of the answer is computed
   similarly with the vector T.
4. Fine-tune BERT and learn S and T along the way.

**References:**

- [BERT](https://arxiv.org/pdf/1810.04805.pdf)
- [SQuAD](https://arxiv.org/abs/1606.05250)


## Setup


In [ ]:
!pip install tokenizers
!pip install transformers

     |████████████████████████████████| 3.3 MB 5.8 MB/s 
     |████████████████████████████████| 3.4 MB 4.7 MB/s 
     |████████████████████████████████| 895 kB 32.3 MB/s 
     |████████████████████████████████| 596 kB 27.3 MB/s 
     |████████████████████████████████| 61 kB 312 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
#importing libraries
import os
import re
import json
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig

max_len = 384
#configuration = BertConfig()  # default parameters and configuration for BERT

from transformers import RobertaTokenizer, TFRobertaForQuestionAnswering, RobertaTokenizerFast, TFRobertaModel
import tensorflow as tf


## Set-up BERT tokenizer


In [ ]:
# Save the slow pretrained tokenizer
slow_tokenizer =RobertaTokenizer.from_pretrained('roberta-base')
save_path = "roberta_base_uncased/"
if not os.path.exists(save_path):
    os.makedirs(save_path)
slow_tokenizer.save_pretrained(save_path)
import tokenizers
# Load the fast tokenizer from saved file

#tokenizer = ByteLevelBPETokenizer(vocab="roberta_base_uncased/vocab.json", lowercase= True)
tokenizer = tokenizers.ByteLevelBPETokenizer(
    vocab='/content/vocab-roberta-base.json', 
    merges='/content/merges-roberta-base.txt', 
    lowercase=True,
    add_prefix_space=True
)
#tokenizer = BertWordPieceTokenizer("roberta_base_uncased/vocab.json", lowercase=True)

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

## Load the data


In [ ]:
train_data_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json"
train_path = keras.utils.get_file("train.json", train_data_url)
eval_data_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json"
eval_path = keras.utils.get_file("eval.json", eval_data_url)


4866048/4854279 [==============================] - 0s 0us/step


## Preprocess the data

1. Go through the JSON file and store every record as a `SquadExample` object.
2. Go through each `SquadExample` and create `x_train, y_train, x_eval, y_eval`.


In [ ]:

class SquadExample:
    def __init__(self, question, context, start_char_idx, answer_text, all_answers):
        self.question = question
        self.context = context
        self.start_char_idx = start_char_idx
        self.answer_text = answer_text
        self.all_answers = all_answers
        self.skip = False

    def preprocess(self):
        context = self.context
        question = self.question
        answer_text = self.answer_text
        start_char_idx = self.start_char_idx

        # Clean context, answer and question
        context = " ".join(str(context).split())
        question = " ".join(str(question).split())
        answer = " ".join(str(answer_text).split())
        #print(answer)
        # Find end character index of answer in context
        end_char_idx = start_char_idx + len(answer)
        if end_char_idx >= len(context):
            self.skip = True
            return

        # Mark the character indexes in context that are in answer
        is_char_in_ans = [0] * len(context)
        for idx in range(start_char_idx, end_char_idx):
            is_char_in_ans[idx] = 1
        '''
        if(True):
          print(context)
          print(answer)
          print(question)
          print(start_char_idx)
          print(end_char_idx)
          print(is_char_in_ans)
          print(sum(is_char_in_ans))
        '''
        # Tokenize context
        tokenized_context = tokenizer.encode(context)
        
        # Find tokens that were created from answer characters
        offsets = []
        idx = 0
        '''
        for t in enumerate(tokenized_context.ids):
            w = tokenizer.decode([t])
            offsets.append((idx, idx+len(w)))
            idx+=len(w)
        print(idx)
        ans_token_idx = []
        for i, (a,b) in enumerate(offsets):
          sm=np.sum(is_char_in_ans[a:b])
          if(sm>0):
            ans_token_idx.append(i)
        '''
        ans_token_idx = []
        for idx, (start, end) in enumerate(tokenized_context.offsets):
            if sum(is_char_in_ans[start:end]) > 0:
                ans_token_idx.append(idx)
        
        if len(ans_token_idx) == 0:
            self.skip = True
            return
        
        # Find start and end token index for tokens from answer
        start_token_idx = ans_token_idx[0]
        end_token_idx = ans_token_idx[-1]

        # Tokenize question
        tokenized_question = tokenizer.encode(question)

        # Create inputs
        input_ids = tokenized_context.ids + tokenized_question.ids[1:]
        token_type_ids = [0] * len(tokenized_context.ids) + [1] * len(
            tokenized_question.ids[1:]
        )
        attention_mask = [1] * len(input_ids)

        # Pad and create attention masks.
        # Skip if truncation is needed
        padding_length = max_len - len(input_ids)
        if padding_length > 0:  # pad
            input_ids = input_ids + ([0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
        elif padding_length < 0:  # skip
            self.skip = True
            return

        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_mask = attention_mask
        self.start_token_idx = start_token_idx
        self.end_token_idx = end_token_idx
        self.context_token_to_char = tokenized_context.offsets


with open(train_path) as f:
    raw_train_data = json.load(f)

with open(eval_path) as f:
    raw_eval_data = json.load(f)

#Creating examples
def create_squad_examples(raw_data):
    squad_examples = []
    for item in raw_data["data"]:
        for para in item["paragraphs"]:
            context = para["context"]
            for qa in para["qas"]:
                question = qa["question"]
                answer_text = qa["answers"][0]["text"]
                all_answers = [_["text"] for _ in qa["answers"]]
                start_char_idx = qa["answers"][0]["answer_start"]
                squad_eg = SquadExample(
                    question, context, start_char_idx, answer_text, all_answers
                )
                squad_eg.preprocess()
                squad_examples.append(squad_eg)
                
    return squad_examples

#Creating tagets
def create_inputs_targets(squad_examples):
    dataset_dict = {
        "input_ids": [],
        "token_type_ids": [],
        "attention_mask": [],
        "start_token_idx": [],
        "end_token_idx": [],
    }
    for item in squad_examples:
        if item.skip == False:
            for key in dataset_dict:
                dataset_dict[key].append(getattr(item, key))
    for key in dataset_dict:
        dataset_dict[key] = np.array(dataset_dict[key])

    x = [
        dataset_dict["input_ids"],
        dataset_dict["token_type_ids"],
        dataset_dict["attention_mask"],
    ]
    y = [dataset_dict["start_token_idx"], dataset_dict["end_token_idx"]]
    return x, y

#training Data
train_squad_examples = create_squad_examples(raw_train_data)
x_train, y_train = create_inputs_targets(train_squad_examples)
print(f"{len(train_squad_examples)} training points created.")

#Testing Data
eval_squad_examples = create_squad_examples(raw_eval_data)
x_eval, y_eval = create_inputs_targets(eval_squad_examples)
print(f"{len(eval_squad_examples)} evaluation points created.")


87599 training points created.
10570 evaluation points created.


In [ ]:
x_train[0].shape

(85915, 384)

In [ ]:
def create_model():
    ## RoBERTa encoder
    encoder = TFRobertaModel.from_pretrained('roberta-base', output_hidden_states = True)

    ## QA Model - Stacking 2 NN and a embedding layer to get better result
    input_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    token_type_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    attention_mask = layers.Input(shape=(max_len,), dtype=tf.int32)
    embedding = encoder(
    input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask, output_hidden_states = True)
    embedding2 = tf.keras.layers.average([embedding[2][-3], embedding[2][-2], embedding[2][-1]])
    start_logits = layers.Dense(1, name="start_logit", use_bias=False)(embedding2)
    start_logits = layers.Flatten()(start_logits)

    end_logits = layers.Dense(1, name="end_logit", use_bias=False)(embedding2)
    end_logits = layers.Flatten()(end_logits)

    start_probs = layers.Activation(keras.activations.softmax)(start_logits)
    end_probs = layers.Activation(keras.activations.softmax)(end_logits)

    model = keras.Model(
        inputs=[input_ids, token_type_ids, attention_mask],
        outputs=[start_probs, end_probs],
    )
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    optimizer = keras.optimizers.Adam(lr=5e-5)
    model.compile(optimizer=optimizer, loss=[loss, loss])
    return model



This code should preferably be run on Google Colab TPU runtime.
With Colab TPUs, each epoch will take 5-6 minutes.


In [ ]:
use_tpu = True
if use_tpu:
    # Create distribution strategy
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    strategy = tf.distribute.TPUStrategy(tpu)

    # Create model
    with strategy.scope():
        model = create_model()
else:
    model = create_model()

model.summary()


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.10.40.210:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.10.40.210:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)
Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 384)]        0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, 384)]        0           []                               
                                                                                                  
 input_5 (InputLayer)           [(None, 384)]        0           []                               
                                                                                                  
 tf_roberta_model_1 (TFRobertaM  TFBaseModelOutputWi  124645632  ['input_4[0][0]',                
 odel)                          thPoolingAndCrossAt               'input_6[0][0]',          

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


## Create evaluation Callback

This callback will compute the exact match score using the validation data
after every epoch.


In [ ]:

f_score_list = []
exact_match_score_list = []

def normalize_text(text):
    text = text.lower()

    # Remove punctuations
    exclude = set(string.punctuation)
    text = "".join(ch for ch in text if ch not in exclude)

    # Remove articles
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    text = re.sub(regex, " ", text)

    # Remove extra white space
    text = " ".join(text.split())
    return text


class FScore(keras.callbacks.Callback):
    def __init__(self, x_eval, y_eval):
        self.x_eval = x_eval
        self.y_eval = y_eval

    def on_epoch_end(self, epoch, logs=None):
        pred_start, pred_end = self.model.predict(self.x_eval)
        print("start f score")
        f_score = 0.0
        eval_examples_no_skip = [_ for _ in eval_squad_examples if _.skip == False]
        count = 0.0
        for idx, (start, end) in enumerate(zip(pred_start, pred_end)):
            squad_eg = eval_examples_no_skip[idx]
            prec = 0.0
            rec = 0.0
            offsets = squad_eg.context_token_to_char
            start = np.argmax(start)
            end = np.argmax(end)
            if start >= len(offsets):
                continue
            pred_char_start = offsets[start][0]
            if end < len(offsets):
                pred_char_end = offsets[end][1]
                pred_ans = squad_eg.context[pred_char_start:pred_char_end]
            else:
                pred_ans = squad_eg.context[pred_char_start:]

            normalized_pred_ans = normalize_text(pred_ans)
            normalized_true_ans_list = [normalize_text(_) for _ in squad_eg.all_answers]
            normalized_pred_ans_tokens = normalized_pred_ans.split()
            count+=1
            temp_f_score = 0
            for normalize_true_ans in normalized_true_ans_list:
              normalize_true_ans_tokens = normalize_true_ans.split()
              if len(normalized_pred_ans_tokens) == 0 or len(normalize_true_ans_tokens) == 0:
                temp_f_score=1
                break
              common_tokens = set(normalized_pred_ans_tokens) & set(normalize_true_ans_tokens)
              # if there are no common tokens then f1 = 0
              if len(common_tokens) == 0:
                continue
              #Precision and Recall calculation
              prec = len(common_tokens) / len(normalized_pred_ans_tokens)
              rec = len(common_tokens) / len(normalize_true_ans_tokens)
              #Picking the max fscore score across the data set
              temp_f_score = max(temp_f_score, 2 * (prec * rec) / (prec + rec))
            f_score+=temp_f_score
        
        #Taking average of max f score over epochs
        f_score = f_score/count
          
        print(f"\nepoch={epoch+1}, f score={f_score:.2f}")
        f_score_list.append(f_score)
        
class ExactMatch(keras.callbacks.Callback):
    """
    Each `SquadExample` object contains the character level offsets for each token
    in its input paragraph. We use them to get back the span of text corresponding
    to the tokens between our predicted start and end tokens.
    All the ground-truth answers are also present in each `SquadExample` object.
    We calculate the percentage of data points where the span of text obtained
    from model predictions matches one of the ground-truth answers.
    """

    def __init__(self, x_eval, y_eval):
        self.x_eval = x_eval
        self.y_eval = y_eval

    def on_epoch_end(self, epoch, logs=None):
        pred_start, pred_end = self.model.predict(self.x_eval)
        count = 0
        eval_examples_no_skip = [_ for _ in eval_squad_examples if _.skip == False]
        for idx, (start, end) in enumerate(zip(pred_start, pred_end)):
            squad_eg = eval_examples_no_skip[idx]
            offsets = squad_eg.context_token_to_char
            start = np.argmax(start)
            end = np.argmax(end)
            if start >= len(offsets):
                continue
            pred_char_start = offsets[start][0]
            if end < len(offsets):
                pred_char_end = offsets[end][1]
                pred_ans = squad_eg.context[pred_char_start:pred_char_end]
            else:
                pred_ans = squad_eg.context[pred_char_start:]

            normalized_pred_ans = normalize_text(pred_ans)
            normalized_true_ans = [normalize_text(_) for _ in squad_eg.all_answers]
            #For exact match
            if normalized_pred_ans in normalized_true_ans:
                count += 1
        #Calcualting exact match
        acc = count / len(self.y_eval[0])
        print(f"\nepoch={epoch+1}, exact match score={acc:.2f}")
        exact_match_score_list.append(acc)



## Train and Evaluate


In [ ]:
exact_match_callback = ExactMatch(x_eval, y_eval)
f_score_callback = FScore(x_eval, y_eval)
model.fit(
    x_train,
    y_train,
    epochs=3,  # For demonstration, 3 epochs are recommended
    verbose=1,
    batch_size=128,
    callbacks=[exact_match_callback, f_score_callback]
)
#Save Model
model.save('./content/drive/model_robert_squad_avg_128.h5')


Epoch 1/3


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond/Identity_24:0' shape=(None,) dtype=int64>, <tf.Tensor 'cond/Identity_32:0' shape=(None,) dtype=int64>]


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond/Identity_24:0' shape=(None,) dtype=int64>, <tf.Tensor 'cond/Identity_32:0' shape=(None,) dtype=int64>]


  6/672 [..............................] - ETA: 3:46 - loss: 9.6829 - activation_2_loss: 4.8871 - activation_3_loss: 4.7958WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0073s vs `on_train_batch_end` time: 9.3667s). Check your callbacks.


672/672 [==============================] - ETA: 0s - loss: 2.5069 - activation_2_loss: 1.3081 - activation_3_loss: 1.1988

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 384) dtype=int64>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 384) dtype=int64>]



epoch=1, exact match score=0.79
start f score

epoch=1, f score=0.88
672/672 [==============================] - 412s 489ms/step - loss: 2.5069 - activation_2_loss: 1.3081 - activation_3_loss: 1.1988
Epoch 2/3
672/672 [==============================] - ETA: 0s - loss: 1.6743 - activation_2_loss: 0.8825 - activation_3_loss: 0.7919
epoch=2, exact match score=0.80
start f score

epoch=2, f score=0.88
672/672 [==============================] - 260s 387ms/step - loss: 1.6743 - activation_2_loss: 0.8825 - activation_3_loss: 0.7919
Epoch 3/3
672/672 [==============================] - ETA: 0s - loss: 1.3435 - activation_2_loss: 0.7129 - activation_3_loss: 0.6305
epoch=3, exact match score=0.81
start f score

epoch=3, f score=0.89
672/672 [==============================] - 260s 387ms/step - loss: 1.3435 - activation_2_loss: 0.7129 - activation_3_loss: 0.6305


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


**Saving Fscore and Exact Match**

In [ ]:
epoch = [1,2,3]

In [ ]:
f_score_list

[0.8813593939195736, 0.8814380912484053, 0.8887388908491257]

In [ ]:
exact_match_score_list

[0.7921686746987951, 0.7960551884959192, 0.8070345899727944]

In [ ]:
import pandas as pd
d = {'exact_match': exact_match_score_list, 'f_score': f_score_list, 'epoch':epoch}

In [ ]:
df = pd.DataFrame(d)

In [ ]:
df

,exact_match,f_score,epoch
0,0.792169,0.881359,1
1,0.796055,0.881438,2
2,0.807035,0.888739,3


In [ ]:
df.to_csv('./content/drive/squad_robert_avg_128.csv', index=False)